In [1]:
using Yao
using YaoPlots
using LinearAlgebra
using Kronecker

[ Info: Precompiling Yao [5872b779-8223-5990-8dd0-5abbb0748c8c] (cache misses: incompatible header (6), dep missing source (2))
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing Yao [5872b779-8223-5990-8dd0-5abbb0748c8c].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
[ Info: Precompiling YaoBase [a8f54c17-34bc-5a9d-b050-f522fe3f755f] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing YaoBase [a8f54c17-34bc-5a9d-b050-f522fe3f755f].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
[ Info: Precompiling LuxurySparse [d05aeea4-b7d4-55ac-b691-9e7fabb0

┌ Warning: Module YaoBlocks with build ID ffffffff-ffff-ffff-0000-04cd86126c0b is missing from the cache.
│ This may mean YaoBlocks [418bc28f-b43b-5e0b-a6e7-61bbc1a2c1df] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:2541
┌ Info: Skipping precompilation due to precompilable error. Importing YaoPlots [32cfe2d9-419e-45f2-8191-2267705d8dbc].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.


In [9]:
# Function to check Equalty up to phase
function is_global_phase_equal(ψ::Matrix{ComplexF64}, ϕ::Matrix{ComplexF64}; atol=1e-10)
    ψ = ψ / norm(ψ)
    ϕ = ϕ / norm(ϕ)
    inner = dot(conj(ϕ), ψ)
    return abs(abs(inner) - 1) < atol
end

is_global_phase_equal (generic function with 2 methods)

In [21]:
xPhase = 1.503123
yPhase = 0.825345 
zPhase = 4.857364 

4.857364

In [5]:
# just to check endianness
state(ArrayReg(bit"00"))

4×1 Matrix{ComplexF64}:
 1.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im

In [4]:
# initialize register 
register = ArrayReg(bit"000")

ArrayReg{2, ComplexF64, Array...}
    active qubits: 3/3
    nlevel: 2

In [6]:
#prepare bell state
bellCirc = chain(3, put(2=>H), control(2, 3=>X))
state1 = register |> bellCirc

ArrayReg{2, ComplexF64, Array...}
    active qubits: 3/3
    nlevel: 2

In [10]:
s1 = state(state1)

8×1 Matrix{ComplexF64}:
 0.7071067811865475 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
 0.7071067811865475 + 0.0im
                0.0 + 0.0im

In [15]:
s1_alternate = reshape([0.707 + 0.000im, 0.000 + 0.000im, 0.000 + 0.000im, 0.707 + 0.000im, 0.000 + 0.000im, 0.000 + 0.000im, 0.000 + 0.000im, 0.000 + 0.000im], :, 1)

8×1 Matrix{ComplexF64}:
 0.707 + 0.0im
   0.0 + 0.0im
   0.0 + 0.0im
 0.707 + 0.0im
   0.0 + 0.0im
   0.0 + 0.0im
   0.0 + 0.0im
   0.0 + 0.0im

In [16]:
is_global_phase_equal(s1, s1_alternate)

false

In [19]:
x = s1/norm(s1)
y = s1_alternate/norm(s1_alternate)
inner = dot(conj(s1), s1_alternate)

0.499924494298889 + 0.0im

In [41]:
# Apply randomized rotations 
#rotCirc = chain(3, put(1=>Rx(xPhase)), put(1=>Ry(yPhase)), put(1=>Rz(zPhase)))
rotCirc = chain(3, put(1=>H))
state2 = state1 |> rotCirc

ArrayReg{2, ComplexF64, Array...}
    active qubits: 3/3
    nlevel: 2

In [42]:
state(state2)

8×1 Matrix{ComplexF64}:
 0.4999999999999999 + 0.0im
 0.4999999999999999 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
 0.4999999999999999 + 0.0im
 0.4999999999999999 + 0.0im

In [10]:
entangleCirc = chain(3, control(1, 2=>X), put(1=>H))
state3 = state2 |> entangleCirc

ArrayReg{2, ComplexF64, Array...}
    active qubits: 3/3
    nlevel: 2

In [11]:
state(state3)

8×1 Matrix{ComplexF64}:
 -0.06988852469142956 + 0.0099499868085135im
 -0.25730436761243536 - 0.6548366622956676im
 -0.06988852469142956 + 0.0099499868085135im
 -0.25730436761243536 - 0.6548366622956676im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im

In [12]:
ket0 = [1, 0]
ket1 = [0, 1]

Id = [[1 0]; [0 1]]

P0 = ket0 * ket0'
P1 = ket1 * ket1'

expandedP0one = P0 ⊗ Id ⊗ Id
expandedP1one = P1 ⊗ Id ⊗ Id 

expandedP0two = Id ⊗ P0 ⊗ Id
expandedP1two = Id ⊗ P1 ⊗ Id 

8×8 Kronecker.KroneckerProduct{Int64, Kronecker.KroneckerProduct{Int64, Matrix{Int64}, Matrix{Int64}}, Matrix{Int64}}:
 0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0
 0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  1

In [13]:
expandedP1two

8×8 Kronecker.KroneckerProduct{Int64, Kronecker.KroneckerProduct{Int64, Matrix{Int64}, Matrix{Int64}}, Matrix{Int64}}:
 0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0
 0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  1

In [14]:
ψ = state(state3)
ψ_1 = expandedP0one * ψ
ψ_2 = expandedP1one * ψ

ψ_3 = expandedP0two * ψ
ψ_4 = expandedP1two * ψ

8×1 Matrix{ComplexF64}:
                  0.0 + 0.0im
                  0.0 + 0.0im
 -0.06988852469142956 + 0.0099499868085135im
 -0.25730436761243536 - 0.6548366622956676im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im